In [1]:
import pandas as pd 
import numpy as np 
import glob
from tqdm import tqdm
from ScrimLog import *
from PeriEventTimeHistogram import *

In [2]:
# FinalStat
scrim = ScrimLog().update_FinalStat_to_csv()
scrim = ScrimLog().update_FinalStat_to_sql()


d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:377: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hero_col['new_col'] = hero_col['Health'] / max_health
d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_player_alive[df_player_alive['IsAlive'] < 1]['IsAlive'] = 0 # replace to 0 if IsAlive < 1. This is required where a player change hero in one second.
File Exported to scrim_finalstat: 20210408_01_O2_Dorado.csv
d:\owl-data\Pr

In [3]:
# 'Shift' Skill
cooldown1 = PETH()
cooldown1.set_search_condition(event_name='Cooldown1%/s', threshold=0.01)
cooldown1.update_PETH_to_sql()

# 'E' Skill
cooldown2 = PETH()
cooldown2.set_search_condition(event_name='Cooldown2%/s', threshold=0.01)
cooldown2.update_PETH_to_sql()

# 'Q' Skill
UU = PETH()
UU.set_search_condition(event_name='UltimatesUsed/s', threshold=1)
UU.update_PETH_to_sql()

# 'R click' Skill
cooldownSecondaryFire = PETH()
cooldownSecondaryFire.set_search_condition(event_name='CooldownSecondaryFire%/s', threshold=0.01)
cooldownSecondaryFire.update_PETH_to_sql()

# 'Ctrl' Skill
cooldownCrouching = PETH()
cooldownCrouching.set_search_condition(event_name='CooldownCrouching%/s', threshold=0.01)
cooldownCrouching.update_PETH_to_sql()

# 'FB'
FB = PETH()
FB.set_search_condition(event_name='FinalBlows/s', threshold=1)
FB.update_PETH_to_sql()

 be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_event_recorder['Timestamp'] = df_event_recorder['Timestamp'].astype(int) # Timestamp 소숫점 자리 버림
d:\owl-data\Project\project 5_scrim stat log\PeriEventTimeHistogram.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_event_recorder['ref_Team'] = ref_team_name
d:\owl-data\Project\project 5_scrim stat log\PeriEventTimeHistogram.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

KeyboardInterrupt: 